In [ ]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np 
import sqlite3
import matplotlib.pyplot as plt
from time import sleep
from apikey import API_KEY

In [ ]:
watcher = LolWatcher(API_KEY)
region = 'na1'

In [ ]:
connection = sqlite3.connect("matches.db")
create_match_table = """CREATE TABLE IF NOT EXISTS match (
id INTEGER PRIMARY KEY,
game_id INTEGER,
team_1 TEXT,
team_2 TEXT,
gold_diff TEXT,
winner TEXT
)"""
cur = connection.cursor()
cur.execute(create_match_table)
insert_match = "INSERT OR IGNORE INTO match (game_id, team_1, team_2, gold_diff, winner) VALUES (?, ?, ?, ?, ?)"

In [ ]:
me = watcher.summoner.by_name(region, "Linom")
me

In [ ]:
challenger_dto = watcher.league.challenger_by_queue(region, "RANKED_SOLO_5x5")
challenger_players = challenger_dto["entries"]

In [ ]:
challenger_names = []
for summoner in challenger_players:
    challenger_names.append(summoner["summonerName"])
challenger_names[0:5]
    

In [ ]:
# User summoner name to get summoner info
def get_summoner_uid(name):
    try:
        summoner = watcher.summoner.by_name(region, name)
    except ApiError:
        return "No summoner found"
    return summoner["puuid"]

In [ ]:
# get gold info for last 10 games
def get_gold_per_match(player_uid):
    match_list = watcher.match.matchlist_by_puuid('americas', player_uid, type="ranked")
    for i in range(len(match_list)):
        game_timeline = watcher.match.timeline_by_match('americas', match_list[i])
        game_details = watcher.match.by_id('americas', match_list[i])
        if game_details["info"]["participants"][0]["win"] == False:
            winner = "Team 1"
        else:
            winner = "Team 2"
        # team_1_win = game_details["info"]["participants"][0]["win"]
        
        gold = {}
        for k in range(len(game_timeline['info']['frames'])):
            gold['{}'.format(k)] = {}
        for value in gold.values():
            for l in range(1, 11):
                value['{}'.format(l)] = 0

        for m in range(len(game_timeline['info']['frames'])):
            for n in range(1, 11):
                gold['{}'.format(m)]['{}'.format(n)] = game_timeline['info']['frames'][m]['participantFrames']['{}'.format(n)]['totalGold']

        team_1_gold = []
        team_2_gold = []
        for key in gold.keys():
            team1 = 0
            team2 = 0
            for o in range(1, 6):
                team1 += gold[key]['{}'.format(o)]
                team2 += gold[key]['{}'.format(o + 5)]
            team_1_gold.append(team1)
            team_2_gold.append(team2)
        frame_size = [val for val in range(len(team_1_gold))]

        gold_diff = (np.array(team_1_gold) - np.array(team_2_gold)).tolist()
        
        # id, game_id, team_1, team_2, gold_diff, winner
        cur.execute(insert_match, (match_list[i], str(team_1_gold), str(team_2_gold), str(gold_diff), winner))
        connection.commit()
        sleep(0.85)
        # return((match_list(i), team_1_gold, team_2_gold, gold_diff) 
        # plt.plot(frame_size, team_1_gold, label="Team 1", marker='o')
        # plt.title('GoldTeam 1')
        # plt.xlabel('Minutes')
        # plt.ylabel('Gold')
        # plt.plot(frame_size, team_2_gold, label="Team 2", marker='o')
        # plt.plot(frame_size, gold_diff, label="Diff", marker='o')
        # plt.legend()
        # plt.show()
 

In [ ]:
for name in challenger_names:
    player_uid = get_summoner_uid(name)
    if player_uid == "No summoner":
        return "No summoner found"
    try:
        get_gold_per_match(player_uid)
    except:
        continue
    

In [ ]:
connection.close()